# RAG + Chain

In [106]:
%%capture --no-stderr
%pip install langchain langchain-openai langchain-openai langchain_chroma langchain-text-splitters langchain_community langchainhub nest_asyncio langchain-text-splitters langchain-openai sentence-transformers

In [107]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

### Tutorial

In [108]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [109]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition refers to the process of breaking down complex tasks into smaller, more manageable steps. This is often achieved through a technique called Chain of Thought (CoT), where models are prompted to "think step by step." This approach not only enhances performance on difficult tasks but also provides insight into the model\'s reasoning process.'

**1. 3개의 블로그 포스팅 본문을 Load: WebBaseLoader 활용**
https://python.langchain.com/v0.2/docs/integrations/document_loaders/web_base/

In [110]:
from langchain_community.document_loaders import WebBaseLoader
import nest_asyncio

nest_asyncio.apply()

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]
loaded_documents = WebBaseLoader(urls).aload()
contents = [x.page_content for x in loaded_documents]
content = "\n".join(contents)

Fetching pages: 100%|##########| 3/3 [00:00<00:00, 16.68it/s]


**2. 불러온 본문을 Split (Chunking) : recursive text splitter 활용**

https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/

In [111]:
import re
cleaned_content = re.sub(r'\n+', '\n', content)

In [112]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
chunked_contents = text_splitter.split_text(cleaned_content)

**3. Chunks 를 임베딩하여 Vector store 저장: openai, chroma 사용**

embedding model 은 "text-embedding-3-small" 사용

embedding: https://python.langchain.com/v0.2/docs/integrations/text_embedding/openai/

vetor store: https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/

In [113]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import chromadb


model = OpenAIEmbeddings(model="text-embedding-3-small")
new_client = chromadb.EphemeralClient()
vectorstore = Chroma.from_texts(
    chunked_contents,
    model,
    client=new_client,
    collection_name="openai_collection"
)

In [114]:
query = "Barack Obama"
docs = vectorstore.similarity_search(query)
print(docs)

[Document(page_content='with "Barack Obama" but leading to toxic output. Given an auditing objective $\\phi: \\mathcal{X}'), Document(page_content='with "Barack Obama" but leading to toxic output. Given an auditing objective $\\phi: \\mathcal{X}'), Document(page_content='with "Barack Obama" but leading to toxic output. Given an auditing objective $\\phi: \\mathcal{X}'), Document(page_content='\\mathbf{y})$ that match certain behavior pattern; such as non-toxic input starting with "Barack')]


**4. User query = ‘agent memory’ 를 받아 관련된 chunks를 retrieve**

https://python.langchain.com/v0.2/docs/how_to/vectorstore_retriever/

https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html#langchain_core.vectorstores.VectorStore.as_retriever

In [120]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

docs = retriever.invoke("what did the president say about ketanji brown jackson?")

**5. User query와 retrieved chunk 에 대해 relevance 가 있는지를 평가하는 시스템 프롬프트 작성: retrieval 퀄리티를 LLM 이 스스로 평가하도록 하고, 관련이 있으면 {‘relevance’: ‘yes’} 관련이 없으면 {‘relevance’: ‘no’} 라고 출력하도록 함. ( JsonOutputParser() 를 활용 )**

RAG 용 프롬프트 작성을 위한 Prompt Hub 활용

https://smith.langchain.com/hub/rlm/rag-prompt

In [148]:
def checkRelevance(user_query, content):
    parser = JsonOutputParser()
    prompt = PromptTemplate(
        temperature="0.2",
        template="Please answer yes or no if {user_query} and {content} are related. Answer template is given as a string in json form, such as 'relevance': 'yes' or 'relevance': 'no'. The result must be a string wrapped in a string.",
        input_variables=["user_query", "content"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    chain = prompt | llm | parser
    return chain.invoke({"user_query": user_query, "content": content})

In [149]:
checkRelevance("what did the president say about ketanji brown jackson?", docs[0].page_content)['relevance']

'no'

In [141]:
from langchain import hub

rlm_prompt = hub.pull("rlm/rag-prompt")
rlm_prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

output JSON formatting

https://python.langchain.com/v0.2/docs/how_to/output_parser_json/#without-pydantic

6. 5 에서 모든 docs에 대해 ‘no’ 라면 디버깅 (Splitter, Chunk size, overlap, embedding model, vector store, retrieval 평가 시스템 프롬프트 등)

7. 5에서 ‘yes’ 라면 질문과 명확히 관련 없는 docs 나 질문 (예: ‘I like an apple’)에 대해서는 ‘no’ 라고 나오는지 테스트 프롬프트 및 평가 코드 작성. 이 때는 관련 없다는 답변 작성

8. ‘yes’ 이고 7의 평가에서도 문제가 없다면, 4의 retrieved chunk 를 가지고 답변 작성
prompt | llm | parser 코드 작성

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

9. 생성된 답안에 Hallucination 이 있는지 평가하는 시스템 프롬프트 작성. LLM이 스스로 평가하도록 하고, hallucination 이 있으면 {‘hallucination’: ‘yes’} 없으면 {‘hallucination’: ‘no’} 라고 출력하도록 함

10. 9 에서 ‘yes’ 면 8 로 돌아가서 다시 생성, ‘no’ 면 답변 생성하고 유저에게 답변 생성에 사용된 출처와 함께 출력 (최대 2번까지 다시 생성)

11. https://applied-llms.org/ 링크는 llm 을 이용해 1년간 개발해 본 팀이 배운 것들을 정리한 아티클 입니다.

저자의 생각을 묻는 질문들에 답하기 위해서

1) Chunking & embedding & storing

2) Load

3) Retrieval

4) Generation

으로 구성된 RAG 코드를 작성하고 아래 예시 질문에 대한 답을 해보세요.

예시)
RAG 에 대한 저자의 생각은 무엇인가?
RAG 와 fine tuning 에 대해 저자는 어떻게 비교하고 있나?
저자가 가장 많은 부분을 할당해 설명하는 개념은 무엇인가?

11 실습을 마치지 못했더라도, https://applied-llms.org/ 아티클은 꼼꼼히 정독해보시기를 추천 드립니다.